# Práctico 1: Ecuación de Hodgkin-Huxley

$$
I=C_{m}{\frac {{\mathrm {d} }V_{m}}{{\mathrm {d} }t}}+{\bar {g}}_{\text{K}}n^{4}(V_{m}-V_{K})+{\bar {g}}_{\text{Na}}m^{3}h(V_{m}-V_{Na})+{\bar {g}}_{l}(V_{m}-V_{l})
$$

$$
{\displaystyle {\frac {dn}{dt}}=\alpha _{n}(V_{m})(1-n)-\beta _{n}(V_{m})n}
$$

$$
{\displaystyle {\frac {dm}{dt}}=\alpha _{m}(V_{m})(1-m)-\beta _{m}(V_{m})m}
$$

$$
{\displaystyle {\frac {dh}{dt}}=\alpha _{h}(V_{m})(1-h)-\beta _{h}(V_{m})h}
$$

Veremos algunas propiedades elementales del modelo de Axón de Hodgkin y Huxley.

## Configuración

Ejecutá las siguientes celdas para configurar el entorno del notebook

In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pylab as plt
from scipy.integrate import odeint
from scipy import stats
import scipy.linalg as lin
import ipywidgets as widgets

### Funciones utilitarias

In [ ]:
def I_inj(t, amp1, dur1, amp2=0, dur2=0, sep=50):
  """Crea una función de corriente inyectada con dos pulsos cuadrados.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    amp1 (float): amplitud del primer pulso, en uA/cm^2.
    dur1 (float): duración del primer pulso, en ms.
    amp2 (float): amplitud del segundo pulso, en uA/cm^2.
    dur2 (float): duración del segundo pulso, en ms.
    sep (float): separación mínima entre los dos pulsos, en ms.

  Returns:
    function: una función I(t) que devuelve la corriente inyectada I(t).
  """
  tmax = np.max(t)
  inicios = np.linspace(0, tmax, 8)
  inicio1 = inicios[1]
  inicio2 = inicio1 + dur1 + sep
  
  def I(t):
    return (amp1 * (t > inicio1) - amp1 * (t > inicio1 + dur1) +
            amp2 * (t > inicio2) - amp2 * (t > inicio2 + dur2))

  return I

def integrar_hh(t, I):
  """Integra el modelo de Hodgkin–Huxley con una corriente inyectada I(t).

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    I (function): función de corriente inyectada I(t) en (µA/cm²).

  Returns:
    V (ndarray): potencial de membrana (mV), mismo shape que `t`.
    m (ndarray): variable de activación de Na⁺, shape como `t`.
    h (ndarray): variable de inactivación de Na⁺, shape como `t`.
    n (ndarray): variable de activación de K⁺, shape como `t`.
    ina (ndarray): corriente de sodio I_Na (µA/cm²).
    ik  (ndarray): corriente de potasio I_K (µA/cm²).
    il  (ndarray): corriente de fuga I_L (µA/cm²).
  """
  # Integramos
  def dALLdt(X, t):
      V, m, h, n = X
      dVdt = (I(t) - I_Na(V, m, h) - I_K(V, n) - I_L(V)) / C_m
      dmdt = alpha_m(V)*(1.0-m) - beta_m(V)*m
      dhdt = alpha_h(V)*(1.0-h) - beta_h(V)*h
      dndt = alpha_n(V)*(1.0-n) - beta_n(V)*n
      return dVdt, dmdt, dhdt, dndt
  X = odeint(dALLdt, [-65, 0.05, 0.6, 0.32], t)

  V = X[:,0]
  m = X[:,1]
  h = X[:,2]
  n = X[:,3]
  ina = I_Na(V, m, h)
  ik = I_K(V, n)
  il = I_L(V)
  
  return V, m, h, n, ina, ik, il

def enmascarar_potenciales_de_accion(t, V, umbral):
  """Detecta cruces ascendentes de umbral en el potencial de membrana.

  Args:
    t (ndarray): vector de tiempos (ms).
    V (ndarray): potencial de membrana (mV).
    umbral (float): umbral de detección (mV).

  Returns:
    ndarray: máscara booleana con True en cada cruce ascendente.
  """
  mascara = (V[1:] > umbral) & (V[:-1] <= umbral)
  mascara = np.concatenate(([False], mascara))
  return mascara

def contar_potenciales_de_accion(mascara):
  """Cuenta los potenciales de acción a partir de una máscara booleana.

  Args:
    mascara (ndarray): máscara booleana con True en cada cruce ascendente.

  Returns:
    int: número de potenciales de acción detectados.
  """
  return np.sum(mascara)

### Funciones de graficado

In [ ]:
def plot_I(t, I):
  plt.figure(figsize=(8, 3))
  plt.plot(t, I)
  plt.xlabel('Tiempo (ms)')
  plt.ylabel('Corriente ($\\mu{A}/cm^2$)')
  plt.ylim(-10, 10)
  plt.show()

def plot_v(t, V, umbral=None, mascara=None):
  fig, axs = plt.subplots(2, 1, figsize=(8, 4), sharex=True)
  axs[0].plot(t, V, 'k')
  axs[0].set_ylabel('V (mV)')
  axs[0].set_ylim(-90, 90)
  if umbral is not None:
    axs[0].axhline(y=umbral)
  if mascara is not None:
    axs[1].eventplot(t[mascara])
    axs[1].set_xlabel("Tiempo (ms)")
    axs[1].set_yticks([])
  plt.tight_layout()
  plt.show()
  
def plot_hh(t, I, V, m, h, n, ina, ik, il):
  """Grafica los resultados de una simulación del axón de Hodgkin-Huxley

  Args:
    t (ndarray): vector 1D con el tiempo (ms).
    V (ndarray): vector 1D con el potencial de membrana (mV).
    m (ndarray): vector 1D con la variable de compuerta de activación del sodio.
    h (ndarray): vector 1D con la variable de compuerta de inactivación del sodio.
    n (ndarray): vector 1D con la variable de compuerta de activación del potasio.
    ina (ndarray): vector 1D con los valores de la corriente de sodio ($I_{Na}$).
    ik (ndarray): vector 1D con los valores de la corriente de potasio ($I_{K}$).
    il (ndarray): vector 1D con los valores de la corriente de fuga ($I_{L}$).
  """
  fig, axes = plt.subplots(4, 1, figsize=(8, 12), gridspec_kw={'height_ratios': [3, 1.2, 3, 3], 'hspace': 0.35})

  # (1) V(t)
  ax = axes[0]
  ax.set_title('Axón de Hodgkin-Huxley')
  ax.plot(t, V, 'k')
  ax.set_ylabel('V (mV)')
  ax.set_ylim(-90, 90)

  # (2) I_inj(t) — más bajo
  ax = axes[1]
  ax.plot(t, I(t), 'k')
  ax.set_xlabel('t (ms)')
  ax.set_ylabel(u'$I_{inj}$ ($\\mu$A/cm$^2$)')
  ax.set_ylim(-10, 10)

  # (3) compuertas vs tiempo
  ax = axes[2]
  ax.plot(t, m, 'r', label='m')
  ax.plot(t, h, 'g', label='h')
  ax.plot(t, n, 'b', label='n')
  ax.set_ylabel('Compuertas')
  ax.legend()

  # (4) corrientes iónicas
  ax = axes[3]
  ax.plot(t, ina, 'c', label='$I_{Na}$')
  ax.plot(t, ik, 'y', label='$I_{K}$')
  ax.plot(t, il, 'm', label='$I_{L}$')
  ax.set_ylabel('Corriente')
  ax.legend()

  plt.show()

def plot_gates(V, m, h, n):
  fig, ax = plt.subplots(figsize=(8, 3))
  ax.plot(V, m, 'r', label='m')
  ax.plot(V, h, 'g', label='h')
  ax.plot(V, n, 'b', label='n')
  ax.set_ylabel('Compuertas vs V')
  ax.set_xlabel('V (mV)')
  ax.legend()
  plt.show()

def plot_fase(t, V, I):
  plt.figure(figsize=(10, 8))

  plt.title('Espacio de Fase')
  plt.xlabel('Voltaje de membrana $V$')
  plt.ylabel(u'Corriente $I$')
  #plt.ylim(-10,1000)
  #plt.xlim=(-80,60)
  plt.plot(V, I)

  plt.show()

## Definición del modelo

### Constantes

In [ ]:
# Capacitancia de membrana específica (µF/cm²).
C_m  = 1.0

# Conductancia máxima de canales de sodio (mS/cm²).
g_Na = 120.0

# Conductancia máxima de canales de potasio (mS/cm²).
g_K  = 36.0

# Conductancia de fuga o leak (mS/cm²).
g_L = 0.05

# Potencial de equilibrio para sodio (mV).
E_Na = 60.0

# Potencial de equilibrio para potasio (mV).
E_K  = -77.0

# Potencial de equilibrio efectivo de la corriente de fuga (mV).
E_L = -54.387  

### Cinética de los canales

Aquí se definen las funciones de apertura ($\alpha$ alfa) y cierre ($\beta$ beta) de las compuertas de los canales de potasio ($m$), y de sodio ($n$ y $h$). Se desprenden de las ecuaciones:

$$
{\displaystyle {\frac {dn}{dt}}=\alpha _{n}(V_{m})(1-n)-\beta _{n}(V_{m})n}
$$

$$
{\displaystyle {\frac {dm}{dt}}=\alpha _{m}(V_{m})(1-m)-\beta _{m}(V_{m})m}
$$

$$
{\displaystyle {\frac {dh}{dt}}=\alpha _{h}(V_{m})(1-h)-\beta _{h}(V_{m})h}
$$

In [ ]:
def alpha_m(V):
  """Tasa α_m de activación de Na⁺ (1/ms).

  Args:
    V (float o ndarray): potencial de membrana en mV.

  Returns:
    float o ndarray: tasa de transición α_m (1/ms).
  """
  return (0.1 * (V + 40.0)) / (1.0 - np.exp(-(V + 40.0) / 10.0))


def beta_m(V):
  """Tasa β_m de desactivación de Na⁺ (1/ms).

  Args:
    V (float o ndarray): potencial de membrana en mV.

  Returns:
    float o ndarray: tasa de transición β_m (1/ms).
  """
  return 4.0 * np.exp(-(V + 65.0) / 18.0)


def alpha_h(V):
  """Tasa α_h de recuperación (inactivación) de Na⁺ (1/ms).

  Args:
    V (float o ndarray): potencial de membrana en mV.

  Returns:
    float o ndarray: tasa de transición α_h (1/ms).
  """
  return 0.07 * np.exp(-(V + 65.0) / 20.0)


def beta_h(V):
  """Tasa β_h de cierre (inactivación) de Na⁺ (1/ms).

  Args:
    V (float o ndarray): potencial de membrana en mV.

  Returns:
    float o ndarray: tasa de transición β_h (1/ms).
  """
  return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))


def alpha_n(V):
  """Tasa α_n de activación de K⁺ (1/ms).

  Args:
    V (float o ndarray): potencial de membrana en mV.

  Returns:
    float o ndarray: tasa de transición α_n (1/ms).
  """
  return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))


def beta_n(V):
  """Tasa β_n de desactivación de K⁺ (1/ms).

  Args:
    V (float o ndarray): potencial de membrana en mV.

  Returns:
    float o ndarray: tasa de transición β_n (1/ms).
  """
  return 0.125 * np.exp(-(V + 65.0) / 80.0)

### Corrientes de membrana

Dependen de la conductancia máxima de los canales ($g$), del grado de apertura (compuertas $m$,$n$ y $k$), y de la diferencia entre el potencial de membrana ($V$) y el potencial de equilibrio ($E$). Se desprenden de la ecuación:

$$
I=C_{m}{\frac {{\mathrm {d} }V_{m}}{{\mathrm {d} }t}}+{\bar {g}}_{\text{K}}n^{4}(V_{m}-V_{K})+{\bar {g}}_{\text{Na}}m^{3}h(V_{m}-V_{Na})+{\bar {g}}_{l}(V_{m}-V_{l})
$$

In [ ]:
def I_Na(V, m, h):
  """Corriente de sodio (Na⁺).
  
  Args:
    V (float o ndarray): potencial de membrana (mV).
    m (float o ndarray): variable de activación de Na⁺.
    h (float o ndarray): variable de inactivación de Na⁺.

  Returns:
    float o ndarray: corriente I_Na (µA/cm²).
  """
  return g_Na * m**3 * h * (V - E_Na)
  
def I_K(V, n):
  """Corriente de potasio (K⁺).

  Args:
    V (float o ndarray): potencial de membrana (mV).
    n (float o ndarray): variable de activación de K⁺.

  Returns:
    float o ndarray: corriente I_K (µA/cm²).
  """
  return g_K * n**4 * (V - E_K)
  
def I_L(V):
  """Corriente de fuga (leak).

  Args:
    V (float o ndarray): potencial de membrana (mV).

  Returns:
    float o ndarray: corriente I_L (µA/cm²).
  """
  return g_L * (V - E_L)

### Integración de las variables

Te dejamos una función disponible llamada `integrar_hh` que integra las variables. Examiná su documentación ejecutando la siguiente celda:

In [ ]:
help(integrar_hh)

## Simulación del modelo

Aquí se implementa la simulación del voltaje de membrana, en función de la corriente entrante, la corriente capacitiva y las corrientes iónicas.
Al final de la simulación, se plotean los resultados.

### Definición de la corriente inyectada

Primero, exploraremos una función que te dejamos disponible llamada `I_inj`. Examiná su documentación ejecutando la siguiente línea:

In [ ]:
help(I_inj)

También podemos ver qué devuelve con el siguiente _widget_ interactivo:

In [ ]:
@widgets.interact(amp1=(-5, 5, 0.1), dur1=(0, 100, 1), amp2=(-5, 5, 0.1), dur2=(0, 100, 1), sep=(0, 200, 1))
def plot_I_inj(amp1=2, dur1=50, amp2=0, dur2=0, sep=50):
  tmax = 400.0
  t = np.arange(0, tmax, 1)
  I = I_inj(t, amp1, dur1, amp2, dur2, sep)
  plot_I(t, I(t))

### Simulación de la corriente inyectada

Ahora, ya podemos simular como responde nuestro axón a diferentes corrientes inyectadas.

In [ ]:
@widgets.interact(amp1=(-5, 5, 0.1), dur1=(0, 100, 1), amp2=(-5, 5, 0.1), dur2=(0, 100, 1), sep=(0, 200, 1))
def simulate(amp1=2, dur1=50, amp2=0, dur2=0, sep=50):
  t = np.arange(0.0, 400.0, 1)
  I = I_inj(t, amp1, dur1, amp2, dur2, sep)
  V, m, h, n, ina, ik, il = integrar_hh(t, I)
  plot_hh(t, I, V,m,h,n,ina,ik,il)

### Visualización de las compuertas

In [ ]:
@widgets.interact(amp=(-5, 5, 0.1), dur=(0, 100, 0.1))
def simulate(amp=2, dur=50):
  t = np.arange(0.0, 100.0, 0.01)
  I = I_inj(t, amp, dur)
  V, m, h, n, ina, ik, il = integrar_hh(t, I)
  plot_gates(V, m, h, n)

### Visualización del espacio de fase

In [ ]:
@widgets.interact(amp=(-5, 5, 0.1), dur=(0, 100, 0.1), corriente=['I_k', 'I_Na', 'I_leak'])
def fase_hh(amp=2, dur=50, corriente='ik'):
  t = np.arange(0.0, 100.0, 0.01)
  I = I_inj(t, amp, dur)
  V, m, h, n, ina, ik, il = integrar_hh(t, I)
  if corriente == 'I_k':
    plot_fase(t, V, ik)
  elif corriente == 'I_Na':
    plot_fase(t, V, ina)
  else:
    plot_fase(t, V, il)

## Relación corriente de entrada vs. frecuencia de disparo

Vamos a simular corrientes de entrada a diferentes amplitudes (intensidades) y evaluar la frecuencia de disparo a para cada valor. Para esto, es necesario antes poder tener una medida de frecuencia de disparo a partir de los resultados de una simulación. Veamos una función utilitaria que enmascara los potenciales de acción a partir de la variable $V$:

In [ ]:
help(enmascarar_potenciales_de_accion)

Ahora miremos en detalle que es lo que devuelve la función:

In [ ]:
t = np.arange(0.0, 200.0, 1)
I = I_inj(t, 10, 150)
V, m, h, n, ina, ik, il = integrar_hh(t, I)
umbral = -20
mascara = enmascarar_potenciales_de_accion(t, V, umbral)

print("Potencial de membrana:")
print(np.round(V, 1)[:10])

print("Máscara:")
print(mascara[:10])

Ahora corré la celda siguiente para ver un gráfico interactivo que te permite seleccionar distintos umbrales:

In [ ]:
@widgets.interact(amp=(0, 20, 0.1), umbral=(-90, 90, 0.1))
def correr_hh(amp, umbral):
  t = np.arange(0.0, 200.0, 1)
  I = I_inj(t, amp, 150)
  V, m, h, n, ina, ik, il = integrar_hh(t, I)
  mascara = enmascarar_potenciales_de_accion(t, V, umbral)
  plot_v(t, V, umbral, mascara)

Solo resta examinar otra función utilitaria que simplemente cuenta los potenciales de acción en una máscara dada:

In [ ]:
help(contar_potenciales_de_accion)

In [ ]:
t = np.arange(0.0, 200.0, 1)
I = I_inj(t, 10, 150)
V, m, h, n, ina, ik, il = integrar_hh(t, I)
umbral = -20
mascara = enmascarar_potenciales_de_accion(t, V, umbral)
frecuencia = contar_potenciales_de_accion(mascara)

print(f"Frecuencia: {frecuencia}")

Ahora sí, tenemos todo listo para simular diferentes amplitudes y anotar la cantidad de disparos que genera cada una en un período de tiempo dado para comparar. Tené en cuenta que vamos a hacer muchas simulaciones en un bucle de código: en cada iteración vamosa correr una simulación completa del modelo de Hodgkin-Huxley. Con `t = np.arange(0, 200, 1)` ya son 200 pasos, y lo vamos a repetir aproximadamente 120 veces (porque `corrientes = np.arange(0, 12, 0.1)`). Eso son unas 24.000 simulaciones de un sistema de 4 variables dinámicas. No es gigante, pero puede sentirse lento.

In [ ]:
umbral = -20
corrientes = np.arange(0, 12, 0.1)
frecuencias = np.zeros_like(corrientes)

for i, amp in enumerate(corrientes):
  t = np.arange(0.0, 200.0, 1)
  I = I_inj(t, amp, 150)
  V, m, h, n, ina, ik, il = integrar_hh(t, I)
  mascara = enmascarar_potenciales_de_accion(t, V, umbral)
  frecuencias[i] = contar_potenciales_de_accion(mascara)
  
plt.scatter(corrientes, frecuencias)
plt.show()